In [2]:
import pandas as pd, numpy as np, os, glob
import neptune.new as neptune
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import auc

def del_empty(lst):
    if isinstance(lst, list):
        return [del_empty(sub) for sub in lst if sub != []]
    return lst

def flatten(t):
    return [item for sublist in t for item in sublist]

# Neptune.ai init
run = neptune.init(
    name = 'Slip and fall holdout set testing.',
    description = 'Logging testing performance on resnet trained model for detecting slip and fall videos.',
    custom_run_id = 'SLIP-3',
    project="Actuate/slip-and-fall-offline-v2",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmYTljNWFlMi01Y2I4LTQwYTctOTNmOS0wNjc2ZjY0NGQ4MzAifQ==", # Ajay's personal API token
    source_files='*.py'
)
run['model_checkpoints/model'].upload('./weights/slip-fall-chosen-weights/yolact_resnet50_432_74900.pth') # Log current model on Neptune

# Create ground truth file
ground_truth = pd.read_csv('./datasets/slip-fall/ground-truths.csv', sep = '\t')
ground_truth['HUMAN DETS'] = np.where(ground_truth['Condition'] == 'Normal', 0, 1)
ground_truth = ground_truth.drop(['Condition'], axis = 1)
ground_truth = ground_truth.rename({'File ID': 'File'}, axis = 'columns')


# Create fused detections dataframe 
basepath = './results/results_v3_transfer_learning/charts/'
confidence_directories = sorted(os.listdir(basepath))
super_list = []
for directory in confidence_directories:
    lists = [] 
    active_directory = os.path.join(basepath, directory)
    files_to_parse = sorted(glob.glob(active_directory + '/*.csv'))
    for file in files_to_parse:
            parsed_confidence = os.path.dirname(file).split('/', 4)[4].split('-')[0:2]
            addition = {'Confidence': '.'.join(parsed_confidence)}
            df = pd.read_csv(file)
            df = df.drop(df.columns[0], axis = 1)
            try:
                output_dictionary = df.groupby(['File']).agg(num_falls = ('Track ID', 'count')).reset_index().to_dict('records')
            except KeyError:
                output_dictionary = [{'File': os.path.basename(file).replace('.csv', '.mp4'), 'num_falls': 0}]
            if len(output_dictionary) == 0:
                output_dictionary = [{'File': os.path.basename(file).replace('.csv', '.mp4'), 'num_falls': 0}]
            else:
                pass
            output_dictionary[0].update(addition)
            lists.append(output_dictionary)
    cleaned = del_empty(lists)
    super_list.append(flatten(cleaned))
df = pd.DataFrame(flatten(super_list))
df = df.rename({'num_falls': 'Model Falls'}, axis = 'columns')
df['MODEL DETS'] = np.where(df['Model Falls'] >= 1, 1, 0)
df = df.merge(ground_truth, on = 'File')
df['Confidence'] = pd.to_numeric(df['Confidence'])
binary_detections = df.copy()
binary_detections.to_csv(os.path.join(basepath, 'fused_results_table.csv'))
run['tables/csv'].upload('./results/results_v3_transfer_learning/charts/fused_results_table.csv') # Log CSV file on Neptune

# Create confusion Matrix
values = []
for conf, dataframe in binary_detections.groupby('Confidence'):
    f1_value = f1_score(dataframe['HUMAN DETS'], dataframe['MODEL DETS'])
    cm = confusion_matrix(dataframe['HUMAN DETS'], dataframe['MODEL DETS'], labels = [0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = np.array([0, 1]))
    f1,ax = plt.subplots(1, figsize = (5,5))
    disp.plot(ax = ax)
    ax.set(title = 'Confusion Matrix @ IOU: CF {}'.format(conf))
    if not os.path.exists('./results/results_v3_transfer_learning/charts/confusion_matrix'):
        os.makedirs('./results/results_v3_transfer_learning/charts/confusion_matrix')
    f1.savefig('./results/results_v3_transfer_learning/charts/confusion_matrix/confusion_matrix_conf_{}.png'.format(conf))
    run['metrics/confusion_matrix'].log(f1) # Upload confusion matricies
    plt.close(f1)
    tn, fp, fn, tp = cm.ravel()
    val = {'CONF': conf, 
            'TN': tn, 
            'FP': fp, 
            'FN': fn, 
            'TP': tp, 
            'F1': f1_value, 
            'TPR': tp / (tp + fn), 
            'P': tp / (tp + fp), 
            'R': tp / (tp + fn), 
            'FPR': fp / (fp + tn)}
    values.append(val)
metrics = pd.DataFrame(values)
metrics.to_csv('./results/results_v3_transfer_learning/charts/summary_statistics.csv')
run['metrics/summary_stats'].upload('./results/results_v3_transfer_learning/charts/summary_statistics.csv')
run['metrics'] = metrics.sort_values(by = 'F1', ascending = False).dropna(inplace = False).head(1).to_dict('records')[0] # Upload best model parameters

# Graph ROC
f2, ax = plt.subplots(1, figsize = (9,7))
ax.plot(metrics['FPR'].values, metrics['TPR'].values, color=np.random.rand(3,), lw=2)
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label = 'YOLACT Base, AUC: {:.2f}'.format(auc(metrics['FPR'].values, metrics['TPR'].values)))
ax.set(xlim = [0.0, 1.0], ylim = [0.0, 1.05], yticks = [i/10.0 for i in range(11)], xticks = [i/10.0 for i in range(11)], xlabel = 'False Positive Rate', ylabel = 'True Positive Rate')
ax.legend(loc = 'lower right')
f2.savefig('./results/results_v3_transfer_learning/charts/roc_auc_curve.png') # Upload all ROC curves.
run['roc_auc_curve'].upload(f2)
plt.close(f2)
run.stop()

https://app.neptune.ai/Actuate/slip-and-fall-offline-v2/e/SLIP-4
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/gpu_memory. Invalid point: 2022-01-12T16:53:44.273Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/cpu. Invalid point: 2022-01-12T16:53:44.273Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/memory. Invalid point: 2022-01-12T16:53:44.273Z


Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 38 operations to synchronize with Neptune. Do not kill this process.
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2022-01-12T16:54:10.072Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2022-01-12T16:54:10.098Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/cpu. Invalid point: 2022-01-12T16:53:54.281Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/memory. Invalid point: 2022-01-12T16:53:54.281Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/gpu_memory. Invalid point: 2022-01-12T16:53:54.281Z


All 38 operations synced, thanks for waiting!


Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/gpu_memory. Invalid point: 2022-01-12T16:54:14.096Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/cpu. Invalid point: 2022-01-12T16:54:14.096Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/memory. Invalid point: 2022-01-12T16:54:14.096Z
